In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, min, max, mean, lit, udf

In [0]:
spark = SparkSession.builder.appName("officeDataProject").getOrCreate()

In [0]:
df = spark.read.options(header=True,inferSchema=True).csv("/FileStore/tables/OfficeDataProject.csv")

In [0]:
df.show(5)

+-----------+-----------------+----------+-----+------+---+-----+
|employee_id|    employee_name|department|state|salary|age|bonus|
+-----------+-----------------+----------+-----+------+---+-----+
|       1000|        Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|  Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
|       1002|Rudolph Barringer|        HR|   LA|  3122| 43| 1445|
|       1003|      Tamra Amber|  Accounts|   AK|  5717| 47| 1291|
|       1004|      Mullan Nitz|Purchasing|   CA|  5685| 34| 1394|
+-----------+-----------------+----------+-----+------+---+-----+
only showing top 5 rows



#### Print the total number of employees in the company

In [0]:
df.select("employee_id").distinct().count()

Out[117]: 1000

#### Print the total number of departments in the company

In [0]:
df.select("department").distinct().count()

Out[118]: 6

#### Print the department names of the company

In [0]:
df.select("department").distinct().show()

+----------+
|department|
+----------+
|     Sales|
|        HR|
|   Finance|
|Purchasing|
| Marketing|
|  Accounts|
+----------+



#### Print the total number of employees in each department

In [0]:
df.groupBy("department").count().show()

+----------+-----+
|department|count|
+----------+-----+
|     Sales|  169|
|        HR|  171|
|   Finance|  162|
|Purchasing|  166|
| Marketing|  170|
|  Accounts|  162|
+----------+-----+



#### Print the total number of employees in each state

In [0]:
df.groupBy("state").count().show()

+-----+-----+
|state|count|
+-----+-----+
|   LA|  205|
|   CA|  205|
|   WA|  208|
|   NY|  173|
|   AK|  209|
+-----+-----+



#### Print the total number of employees in each state in each department

In [0]:
df.groupBy("state", "department").count().show(5)

+-----+----------+-----+
|state|department|count|
+-----+----------+-----+
|   CA|     Sales|   42|
|   CA| Marketing|   33|
|   NY|  Accounts|   34|
|   NY|     Sales|   27|
|   CA|   Finance|   35|
+-----+----------+-----+
only showing top 5 rows



#### Print the minimum and maximum salaries in each department and sort salaries in ascending order

In [0]:
df.groupBy("department").agg(min("salary").alias("min_salary"),max("salary").alias("max_salary")).sort(col("min_salary"),col("max_salary")).show()

+----------+----------+----------+
|department|min_salary|max_salary|
+----------+----------+----------+
|   Finance|      1006|      9899|
|  Accounts|      1007|      9890|
|        HR|      1013|      9982|
| Marketing|      1031|      9974|
|     Sales|      1103|      9982|
|Purchasing|      1105|      9985|
+----------+----------+----------+



#### Print the names of employees working in NY state under Finance department whose bonuses are greater than the average bonuses of employees in NY state

In [0]:
avg_bonus_df = df.groupBy("state").agg(mean("bonus").alias("avg_bonus")).filter(col("state") == "NY")
avg_bonus = avg_bonus_df.collect()[0]["avg_bonus"]

In [0]:
df.select("employee_name","bonus").filter((col("state")=="NY") & (col("department")=="Finance")).filter(col("bonus") > (avg_bonus)).show(5)

+--------------------+-----+
|       employee_name|bonus|
+--------------------+-----+
|       Vivan Sifford| 1261|
|      Herder Gallman| 1402|
|          Nena Rocha| 1647|
|       Leif Lemaster| 1782|
|Ellingsworth Meli...| 1358|
+--------------------+-----+
only showing top 5 rows



#### Raise the salaries $500 of all employees whose age is greater than 45

In [0]:
def inc(x,y):
    if x > 45:
        return y+500
    else:
        return y

inc_udf = udf(lambda x,y : inc(x,y))

df.withColumn("salary", inc_udf(col("age"), col("salary"))).show(2)

+-----------+---------------+----------+-----+------+---+-----+
|employee_id|  employee_name|department|state|salary|age|bonus|
+-----------+---------------+----------+-----+------+---+-----+
|       1000|      Nitz Leif| Marketing|   CA|  6131| 26|  543|
|       1001|Melissia Dedman|   Finance|   AK|  4027| 43| 1290|
+-----------+---------------+----------+-----+------+---+-----+
only showing top 2 rows



#### Create DF of all those employees whose age is greater than 45 and save them in a file

In [0]:
forty_five = df.filter(col("age") > 45)
forty_five.write.mode("overwrite").csv("/FileStore/tables/output/")